# Feb 24, 28, 2025: spherical rois
yale space, and ccfv2 space

In [1]:
import csv 
import os
import json
import numpy as np
import pandas as pd
import dill as pickle
import ants
import nibabel as nib
from tqdm import tqdm
from glob import glob
from scipy import ndimage

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

np.random.seed(args.SEED)

In [3]:
# RAS_PARCELS_file = f'/home/govindas/mouse_dataset/parcels/type-spatial_size-225_symm-True_braindiv-whl_nrois-162_desc-parcels.nii.gz'
# RAS_PARCELS_file = f'/home/govindas/mouse_dataset/parcels/type-spatial_size-112_symm-True_braindiv-whl_nrois-334_desc-parcels.nii.gz'
# RAS_PARCELS_file = f'/home/govindas/mouse_dataset/parcels/type-spatial_size-168_symm-True_braindiv-whl_nrois-216_desc-parcels.nii.gz'
RAS_PARCELS_file = f'/home/govindas/mouse_dataset/parcels/type-spatial_size-140_symm-True_braindiv-whl_nrois-264_desc-parcels.nii.gz'
ras_parcels_img = ants.image_read(RAS_PARCELS_file)
ras_parcels = ras_parcels_img.numpy()
ras_parcels.shape

(60, 81, 46)

In [4]:
ras_parcels_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [5]:
RAS_TEMPLATE_file = f'/home/govindas/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
ras_template_img = ants.image_read(RAS_TEMPLATE_file)
LPI_TEMPLATE_file = f'/home/govindas/mouse_dataset/gabe_symmetric_N162/Symmetric_N162_0.20.nii.gz'
lpi_template_img = ants.image_read(LPI_TEMPLATE_file)

In [6]:
ras_template_img, lpi_template_img

(ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (60, 81, 46)
 	 Spacing    : (0.2, 0.2, 0.2)
 	 Origin     : (18.2, 2.8, -7.8)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.],
 ANTsImage (RAI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (60, 46, 81)
 	 Spacing    : (0.2, 0.2, 0.2)
 	 Origin     : (6.4, -6.2, -7.8)
 	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.])

In [7]:
def reorient(ras_img, lpi_img):
    ras = ras_img.numpy()
    ras_tx = np.transpose(ras, (0, 2, 1))
    ras_tx = ras_tx[::-1, ::-1, :]
    return lpi_img.new_image_like(ras_tx)

In [8]:
ras_parcels_img = ants.resample_image_to_target(
    image=ras_parcels_img, 
    target=ras_template_img,
    interp_type='genericLabel',
)
lpi_parcels_img = reorient(ras_parcels_img, lpi_template_img)
lpi_parcels_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 46, 81)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (6.4, -6.2, -7.8)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [9]:
np.unique(lpi_parcels_img.numpy())

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [10]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.system(f'mkdir -p {PARCELS_path}')

0

In [11]:
args.source = 'spatial'
args.space = 'yale'
args.brain_div = 'whl'
args.num_rois = 264 #216 #334 #162
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-spatial_space-yale_braindiv-whl_nrois-264_res-200'

In [12]:
lpi_parcels_img.to_file(
    f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
)

In [13]:
# put the template in the PARCELS_path
TEMP_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
template_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'

lpi_template_img.to_file(
    template_file
)

---
run the below when Francesca shares the transformed parcellation

In [16]:
args.source = 'spatial'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 216 #264 #216 #168 #334 #162
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-spatial_space-ccfv2_braindiv-whl_nrois-216_res-200'

In [17]:
PARCELS_file = f'{BASE_path}/parcellation/space-allen/{PARC_DESC}_desc-parcels.nii'
parcels_img = nib.load(PARCELS_file)

nib.save(
    parcels_img, 
    f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
)